<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Clustering + Regresión lineal

Ejemplo de aumentar los datos con clustering<br>
v1.1

### Objetivos: 
*  Construir modelos con GaussianMixture y Regresión Lineal empleando el dataset "FuelConsumptionCo2".
*  Comparar los resultados de los modelos GaussianMixture y Regresión Lineal en el dataset "FuelConsumptionCo2".

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### Código de descarga del dataset

In [ ]:
if os.access('FuelConsumptionCo2.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv > FuelConsumptionCo2.csv
    else:
        !wget FuelConsumptionCo2.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv

### `FuelConsumption.csv`:
El dataset **`FuelConsumption.csv`** contiene el consumo específico por tipo de vehículo y la emisión estimado de dioxido de carbono (Co2) de nuevos vehículos que son venidos en canada.<br> [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** --> ejemplo 2014
- **MAKE** --> ejemplo Acura
- **MODEL** --> ejemplo ILX
- **VEHICLE CLASS** --> ejemplo SUV
- **ENGINE SIZE** --> ejemplo 4.7
- **CYLINDERS** --> ejemplo 6
- **TRANSMISSION** --> ejemplo A6
- **FUEL CONSUMPTION in CITY(L/100 km)** --> ejemplo 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** --> ejemplo 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** --> ejemplo 9.2
- **CO2 EMISSIONS (g/km)** --> ejemplo 182

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Realizar una inspeccion del dataset en búsqueda de elementos faltantes
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.
df = pd.read_csv("FuelConsumptionCo2.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas
df.head()

In [ ]:
# Se hace una copia
df1 = df.copy()

# Cambiando los nombres de las columnas
df1.columns = ['AÑO', 'MARCA', 'MODELO', 'CLASE_VEHICULO', 'TAMAÑO_MOTOR', 'CILINDROS', 'TRANSMISION', 'TIPO_COMBUSTIBLE', 'CONSUMO_COMBUSTIBLE_CIUDAD', 'CONSUMO_COMBUSTIBLE_HWY', 'CONSUMO_COMBUSTIBLE_COMB', 'CONSUMO_COMBUSTIBLE_COMB_MPG', 'EMISIONES_CO2']

# Muestra las 5 primeras filas
df1.head()

## Fin de la limpieza
Se finalizó la limpieza, no hay datos mal cargados o incompletos en este dataset

In [ ]:
# Cantidad de filas y columnas con shape
# En la ubicación 0 corresponde a las filas
print('Cantidad de datos en observacion:', df1.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Analizaremos como se compartan algunos atributos de entrada contra el objetivo (las emisiones, el Co2)
# sns, alias de Seaborn
# data, indicar todos los datos del DataFrame
# x_vars, indicar las columnas a representar en el eje x
# y_vars, indicar la columna a relacionar
# height=5, altura de los gráficos
# diag_kind=None, para indicar el tipo de diagonal, al ser None, significa ninguna.
pp = sns.pairplot(data=df1, x_vars=['CILINDROS', 'TAMAÑO_MOTOR', 'CONSUMO_COMBUSTIBLE_COMB'], y_vars=['EMISIONES_CO2'], diag_kind=None, height=5)
plt.show()


Analizaremos la relación entre le consumo de combustible (FUELCONSUMPTION_COMB) y la emisión de CO2

Se puede observar que está centrada a la izquierda la distribución y no hay outliers, por lo que no habrá problema. El hecho de que la distribución esté marcada a la izquierda querra decir que hay menos datos para analizar de alta cilindrada.

In [ ]:
# Representación gráfica del consumo de combustible y las emisiones CO2
# a través de un gráfico de dispersión.
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df1['CONSUMO_COMBUSTIBLE_COMB'], y=df1['EMISIONES_CO2'], color='darkViolet', ax=ax)
ax.grid('dashed')
plt.show()

# Entrenar modelo (solo regresión lineal)
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
# Para los valores de "X" se emplea doble corchete porque necesita la matriz de dos dimensiones.
X = df1[['CONSUMO_COMBUSTIBLE_COMB']].values

# Para los valores de "y" se emplea un corchete porque necesita la matriz de una dimensiones
y = df1['EMISIONES_CO2'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
# Se importa la herramienta de sklearn.model_selectionl como train_test_split
from sklearn.model_selection import train_test_split

# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
# Ojo! Los dataset de train y test son array numpy
# Se importa la herramienta de la libreria  train_test_split()
# Necesita los valores de X e y
# test_size=0.3, permite indicar el porcentaje de valores para evaluar, equivalente a un 30%
# random_state=42,  es un número fijo que utilizan comunmente en documentación, significa que para cada ejecución del algoritmo se genere nuevos valores aleatorios
# y los conjuntos de datos de entrenamiento y pruebas serán diferentes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Crear un modelo de regresión lineal

In [ ]:
# Importar herramienta
from sklearn.linear_model import LinearRegression

# Crear objeto lr, a partir de la clase LinearRegression()
lr = LinearRegression()

# Se entrena al modelo
lr.fit(X_train, y_train)

# Se obtienen las predicciones en y_hat
y_hat= lr.predict(X_test)

In [ ]:
# Se representa graficamente los datos de las columnas df['FUELCONSUMPTION_COMB'] y df['CO2EMISSIONS'] en un scatterplot
# También las predicciones con los datos de evaluación.
# X_test[:, 0], ajusta en una sola matriz todos los valores de X_test
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df1['CONSUMO_COMBUSTIBLE_COMB'], y=df['EMISIONES_CO2'], color='darkCyan', ax=ax, label='data')
ax = sns.scatterplot(x=X_test[:, 0], y=y_hat, color='darkViolet', ax=ax, label='linear')
ax.grid('dashed')
ax.legend()
plt.show()

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Se calcula la métrica de r2_score
from sklearn.metrics import r2_score
score = r2_score(y_test, y_hat)
print(f"Coeficiente de determinación: {score:.2f}")

# Entrenar modelo (clustering + regresión)
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

Separar las rectas en 3 conjuntos (clusters) distintos utilizando algún algoritmo de segmentación

In [ ]:
# Copia
df2 = df1.copy()

In [ ]:
#Librerias a implementar
from sklearn.mixture import GaussianMixture


# Se selecciona las columnas relacionadas como: 'CONSUMO_COMBUSTIBLE_COMB', 'EMISIONES_CO2'
# De esas columnas sólo se necesita los valores, por eso se usa .values
X_clustering = df2[['CONSUMO_COMBUSTIBLE_COMB', 'EMISIONES_CO2']].values

# Se crea el objeto gmm a partir de la clase GaussianMixture
# Donde se especifica los siguientes parámetros:
# n_components=3, cantidad de grupos a formar, son 3, para separar los datos por la 3 rectas del gráfico.
# covariance_type, cadena que describe el tipo de parámetros de covarianza a utilizar
# Es 'full', porque cada componente tiene su propia matriz de covarianza general.
gmm = GaussianMixture(n_components=3, covariance_type='full')

# Se entrena con fit, los valores de X_gaussian_mixture
gmm.fit(X_clustering)

# Los resultados de la agrupación se guardan en el DataFrame df2, en la columna 'grupo_consumo'
df2["grupo_consumo"] = gmm.predict(X_clustering)

# Observar la relación entre el 'CONSUMO_COMBUSTIBLE_COMB' y 'EMISIONES_CO2'
# pero ahora coloreando con "hue" los puntos según su clase de "grupo_consumo"
sns.scatterplot(data=df2, x='CONSUMO_COMBUSTIBLE_COMB', y='EMISIONES_CO2', hue='grupo_consumo', palette='bright')
plt.show()

###Realizar una regresión lineal por cada cluster definido

In [ ]:
# Determina cuántos clusters están definidos en la columna 'grupo_consumo'
n_clusters = df2['grupo_consumo'].unique()

# Se guarda cada objeto "lr" creado en cada iteración
lr_models = []

# Bucle que itera la cantidad de veces que indiquen los grupos conformados.
# En este caso 3 grupos, de las tres rectas encontrados con GaussianMixture
for cluster in n_clusters:

    # Verificar cada fila de la columna 'grupo_consumo' es igual al número del cluster que se está iterando
    df_cluster = df2[df2['grupo_consumo'] == cluster]

    # Se obtienen los valores para X_clust e y_clust
    X_clust = df_cluster[['CONSUMO_COMBUSTIBLE_COMB']].values
    y_clust = df_cluster['EMISIONES_CO2'].values

    # Se hace la separación de los datos a entrenar y evaluar
    X_train_clust, X_test_clust, y_train_clust, y_test_clust = train_test_split(X_clust, y_clust, test_size=0.3, random_state=42)

    # Se arma el modelo de regresión lineal
    # Se crea el objeto lr a partir de la clase LinearRegression()
    # Se entrena con fit
    # Se almacenan los resultados en y_hat_clust
    lr = LinearRegression()
    lr.fit(X_train_clust, y_train_clust)
    y_hat_clust = lr.predict(X_test_clust)

    # Se calcula la métrica r2_score
    score = r2_score(y_test_clust, y_hat_clust)
    print(f"Coeficiente de determinación: {score:.2f} para el cluster", cluster)

    # Se guarda el objeto de cada iteración en  lr_models
    lr_models.append(lr)



# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Crear el espacio de dibujo y el espacio para el gráfico.
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()

# Observar la relación entre el 'CONSUMO_COMBUSTIBLE_COMB' y 'EMISIONES_CO2'
# pero ahora coloreando con "hue" los puntos según su clase de "grupo_consumo"
ax = sns.scatterplot(x=df2['CONSUMO_COMBUSTIBLE_COMB'], y=df['CO2EMISSIONS'], color='darkBlue', ax=ax, label='data')

# Bucle que itera los objetos "lr" de cada regresión lineal guardados en "lr_models"
for lr_model in lr_models:
    # Valores de  X_sorted ordenados
    X_sorted = np.asanyarray(sorted(X))

    # y_pred, almacena las predicciones (A qué grupo pertenece cada dato).
    y_pred = lr_model.predict(X_sorted)

    # Representa un gráfico que mostrará 3 líneas de predicción y los datos que representa.
    ax = sns.lineplot(x=X_sorted[:, 0], y=y_pred, ax=ax, label='linear+cluster')

# Grilla de fondo
ax.grid('dashed')

# Activación de leyenda
ax.legend()

# Mostrar la figura
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Este dataset no tiene absolutamente nada de "ruido" o imperfeccciones, por lo que con solo haber realizado múltiples rectas en vez de una con la ayuda de la segmentación previa se obtuvo un resultado 100% perfecto. Con este ejemplo alumno ya está listo para poder realizar su proyecto completo de estimación de alquileres si así lo deseara.